In [1]:
from __future__ import print_function, division
import time
from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from six import iteritems
import warnings
import nilmtk as ntk
from os.path import join
import seaborn as sns;   sns.set()
warnings.filterwarnings('ignore')
%matplotlib inline

rcParams['figure.figsize'] = (13, 6)

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.disaggregate import CombinatorialOptimisation



In [2]:


h5_path = '/Users/Dhruvin suthar/Desktop/Thesis work/test_of_database/DRED/DRED' 
dred_train = ntk.DataSet(join(h5_path,'DRED.h5'))
dred_test = ntk.DataSet(join(h5_path,'DRED.h5'))
print("for training: loaded " + str(len(dred_train.buildings)) + ' buildings')
print("for test :loaded " + str(len(dred_test.buildings)) + ' buildings')

for training: loaded 1 buildings
for test :loaded 1 buildings


In [ ]:
dred_train.set_window(start=None, end=None)
dred_test.set_window(start=None, end='2015-07-20 00:00:00')


dred_train_elec = dred_train.buildings[1].elec
dred_test_elec = dred_test.buildings[1].elec

In [ ]:
dred_train_elec.get_labels([1,2,3,4,5,6,7,8,9,10,11,12,13])

In [ ]:
dred_train_elec.plot()

In [ ]:
dred_test_elec.mains().plot()

In [ ]:
fridge_meter = dred_train_elec['fridge']
fridge_meter.plot()

In [ ]:
fridge_df = next(fridge_meter.load())
fridge_df.head()

In [ ]:
mains = dred_train_elec.mains()
mains.plot()

In [ ]:
mains_df = next(mains.load())
mains_df.head()

In [ ]:
train_elec = dred_train_elec.submeters()
train_elec

In [ ]:
train_elec.plot()

In [ ]:
start = time.time()
co = ntk.disaggregate.CombinatorialOptimisation()
co.train(train_elec, sample_period=30)
end = time.time()
print("Runtime =", end-start, "seconds.")


In [ ]:
dred_train_elec.get_labels([1,2,3,4,5,6,7,8,9,10,11,12,13])

In [ ]:
dred_test_elec.mains()

In [ ]:

pred = {}
gt= {}

for i, chunk in enumerate(dred_test_elec.mains().load(sample_period=60)):
    chunk_drop_na = chunk.dropna()
    pred[i] = co.disaggregate_chunk(chunk_drop_na)
    gt[i]={}
    print(gt[i])
    for meter in dred_test_elec.submeters().meters:
        # Only use the meters that we trained on (this saves time!)    
        gt[i][meter] = next(meter.load(sample_period=60))
    gt[i] = pd.DataFrame({k:v.squeeze() for k,v in iteritems(gt[i]) if len(v)}, index=next(iter(gt[i].values())).index).dropna()

In [ ]:
# If everything can fit in memory
gt_overall = pd.concat(gt)

gt_overall.index = gt_overall.index.droplevel()
pred_overall = pd.concat(pred)
pred_overall.index = pred_overall.index.droplevel()

# Having the same order of columns
gt_overall = gt_overall[pred_overall.columns]

In [ ]:
#Intersection of index
gt_index_utc = gt_overall.index.tz_convert("UTC")
pred_index_utc = pred_overall.index.tz_convert("UTC")
common_index_utc = gt_index_utc.intersection(pred_index_utc)

In [ ]:
local_timezone = dred_train.metadata['timezone']

In [ ]:
common_index_local = common_index_utc.tz_convert(local_timezone)

In [ ]:
gt_overall = gt_overall.ix[common_index_local]
pred_overall = pred_overall.ix[common_index_local]

In [ ]:
type(gt_overall)

In [ ]:
import pandas as pd
 
for col in gt_overall.columns: 
    print(col) 

In [ ]:
appliance_labels =dred_train_elec.get_labels([1,2,3,4,5,6,7,8,9,10,11,12]) #[m for m in gt_overall.columns.values]
gt_overall.columns = appliance_labels
pred_overall.columns = appliance_labels

In [ ]:
pred_overall.head()

In [ ]:
pred_overall["Television"].head(1000).plot(label="Pred")
gt_overall["Television"].head(1000).plot(label="GT")
plt.legend()

In [ ]:
pred_overall["Fan"].head(5000).plot(label="Pred")
gt_overall["Fan"].head(5000).plot(label="GT")
plt.legend()

In [ ]:
pred_overall["Television"].head(1000).plot(label="Pred")
gt_overall["Television"].head(1000).plot(label="GT")
plt.legend()

In [ ]:
pred_overall["Fridge"].head(5000).plot(label="Pred")
gt_overall["Fridge"].head(5000).plot(label="GT")
plt.legend()

In [ ]:
pred_overall["Electric heating element"].head(1000).plot(label="Pred")
gt_overall["Electric heating element"].head(1000).plot(label="GT")
plt.legend()

In [ ]:

pred_overall["Oven"].head(1000).plot(label="Pred")
gt_overall["Oven"].head(1000).plot(label="GT")
plt.legend()

In [ ]:

pred_overall["Unknown"].head(10000).plot(label="Pred")
gt_overall["Unknown"].head(10000).plot(label="GT")
plt.legend()

In [ ]:

pred_overall["Washing machine"].head(10000).plot(label="Pred")
gt_overall["Washing machine"].head(10000).plot(label="GT")
plt.legend()

In [ ]:

pred_overall["Microwave"].head(10000).plot(label="Pred")
gt_overall["Microwave"].head(10000).plot(label="GT")
plt.legend()

In [ ]:


pred_overall["Toaster"].head(10000).plot(label="Pred")
gt_overall["Toaster"].head(10000).plot(label="GT")
plt.legend()

In [ ]:



pred_overall["Sockets"].head(10000).plot(label="Pred")
gt_overall["Sockets"].head(10000).plot(label="GT")
plt.legend()

In [ ]:

pred_overall["Site meter"].head(10000).plot(label="Pred")
gt_overall["Site meter"].head(10000).plot(label="GT")
plt.legend()

In [ ]:
start = time.time()
from nilmtk.disaggregate import fhmm_exact
fhmm = fhmm_exact.FHMM()
# Note that we have given the sample period to downsample the data to 1 minute. 
# If instead of top_5 we wanted to train on all appliance, we would write 
# fhmm.train(train_elec, sample_period=60)
fhmm.train(train_elec, sample_period=60)
end = time.time()
print("Runtime =", end-start, "seconds.")
